# Attention Mechanism from Scratch

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adiel2012/deep-learning-abc/blob/main/attention_from_scratch.ipynb)

This notebook implements the attention mechanism using **only low-level tensor operations** — no `nn.Module`, no `nn.Linear`, no high-level wrappers.

We build everything from raw matrix multiplications and element-wise operations so you can see exactly what happens at each step.

> **Companion notebook:** [attention_with_pytorch.ipynb](https://colab.research.google.com/github/adiel2012/deep-learning-abc/blob/main/attention_with_pytorch.ipynb) — same content using `nn.Module`.

In [ ]:
# Install dependencies (Colab already has torch, but this ensures compatibility)
!pip install torch matplotlib -q

## 0. Mathematical Foundations

Before writing any code, let's build up the math that attention relies on.

---

### 0.1 Vectors and Dot Products

A **vector** $\mathbf{a} \in \mathbb{R}^d$ is an ordered list of $d$ real numbers. The **dot product** of two vectors measures their alignment:

$$\mathbf{a} \cdot \mathbf{b} = \sum_{i=1}^{d} a_i \, b_i = \|\mathbf{a}\| \, \|\mathbf{b}\| \, \cos\theta$$

where $\theta$ is the angle between them. Key intuitions:
- **Positive & large**: vectors point in a similar direction (high similarity)
- **Near zero**: vectors are roughly orthogonal (unrelated)
- **Negative**: vectors point in opposite directions

In attention, the dot product $q_i \cdot k_j$ measures **how relevant** token $j$ is to token $i$.

---

### 0.2 Matrix Multiplication

If $X \in \mathbb{R}^{n \times d}$ and $W \in \mathbb{R}^{d \times m}$, then:

$$(XW)_{ij} = \sum_{k=1}^{d} X_{ik} \, W_{kj}$$

Each row of the result is a **linear combination** of the columns of $W$, weighted by the corresponding row of $X$. This is how we project embeddings into Q, K, V spaces — it's a learned linear transformation.

When we compute $Q K^T$, we're doing all pairwise dot products at once:

$$(Q K^T)_{ij} = \mathbf{q}_i \cdot \mathbf{k}_j$$

So row $i$ of the resulting matrix contains the similarity of query $i$ with every key.

---

### 0.3 The Softmax Function

Softmax converts a vector of arbitrary real numbers into a **probability distribution** (non-negative, sums to 1):

$$\text{softmax}(\mathbf{z})_i = \frac{e^{z_i}}{\sum_{j=1}^{n} e^{z_j}}$$

Properties:
- **Output range**: each value is in $(0, 1)$, and they sum to exactly $1$
- **Monotonic**: larger inputs get larger probabilities
- **Sharpness**: as differences between inputs grow, softmax approaches a one-hot vector (winner-take-all)

**Numerical stability trick**: since $e^{z_i}$ can overflow for large $z_i$, we use the identity:

$$\text{softmax}(\mathbf{z})_i = \text{softmax}(\mathbf{z} - c)_i \quad \text{for any constant } c$$

Setting $c = \max_j z_j$ keeps the exponents in a safe range.

---

### 0.4 Why Scale by $\sqrt{d_k}$?

If the entries of $\mathbf{q}$ and $\mathbf{k}$ are i.i.d. with mean 0 and variance 1, then:

$$\text{Var}(\mathbf{q} \cdot \mathbf{k}) = \text{Var}\!\left(\sum_{i=1}^{d_k} q_i k_i\right) = d_k$$

So the dot product's variance **grows linearly** with $d_k$. For large $d_k$, the scores become large in magnitude, pushing softmax into saturation (near-zero gradients).

Dividing by $\sqrt{d_k}$ normalizes the variance back to 1:

$$\text{Var}\!\left(\frac{\mathbf{q} \cdot \mathbf{k}}{\sqrt{d_k}}\right) = \frac{d_k}{d_k} = 1$$

---

### 0.5 Attention as a Weighted Average

The final attention output for token $i$ is:

$$\text{output}_i = \sum_{j=1}^{n} \alpha_{ij} \, \mathbf{v}_j$$

where $\alpha_{ij} = \text{softmax}\!\left(\frac{\mathbf{q}_i \cdot \mathbf{k}_j}{\sqrt{d_k}}\right)_j$ are the attention weights.

This is a **convex combination** of the value vectors — the output lives in the convex hull of the $\mathbf{v}_j$'s. Tokens with higher similarity scores contribute more to the output.

---

### 0.6 Linear Projections: Why Q, K, V?

Raw embeddings aren't optimized for measuring relevance. The learnable matrices $W_Q, W_K, W_V$ project each token into three roles:

| Matrix | Role | Analogy |
|--------|------|--------|
| $W_Q$ | **Query** — "what am I looking for?" | A database query |
| $W_K$ | **Key** — "what do I contain?" | A database index |
| $W_V$ | **Value** — "what information do I carry?" | The actual data |

The dot product $\mathbf{q}_i \cdot \mathbf{k}_j$ computes **relevance** (query matches key), and the weighted sum over $\mathbf{v}_j$ retrieves **content** from the relevant tokens.

---

Now let's implement all of this step by step.

In [ ]:
import torch
import math
import matplotlib.pyplot as plt

torch.manual_seed(42)

# Use GPU if available (Colab: Runtime > Change runtime type > GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 1. Setup: Fake Token Embeddings

Imagine a sentence with 4 tokens, each represented by an 8-dimensional embedding vector.

| Index | Token   |
|-------|---------|
| 0     | "The"   |
| 1     | "cat"   |
| 2     | "sat"   |
| 3     | "down"  |

In [ ]:
seq_len = 4    # number of tokens
d_model = 8    # embedding dimension

# Random embeddings simulating the output of an embedding layer
X = torch.randn(seq_len, d_model, device=device)
print("Input X shape:", X.shape)
print(X)

## 2. Projecting to Query, Key, Value

Attention doesn't operate on raw embeddings directly. We first project `X` into three different spaces using weight matrices:

$$Q = X \cdot W_Q$$
$$K = X \cdot W_K$$
$$V = X \cdot W_V$$

Each weight matrix has shape `(d_model, d_k)`. We use raw `torch.randn` to create them — no `nn.Linear`.

In [ ]:
d_k = 6  # dimension of queries and keys
d_v = 6  # dimension of values

# Weight matrices — raw parameters, no nn.Module
W_Q = torch.randn(d_model, d_k, device=device) * 0.1
W_K = torch.randn(d_model, d_k, device=device) * 0.1
W_V = torch.randn(d_model, d_v, device=device) * 0.1

# Project: simple matrix multiplication
Q = X @ W_Q   # (seq_len, d_k)
K = X @ W_K   # (seq_len, d_k)
V = X @ W_V   # (seq_len, d_v)

print("Q shape:", Q.shape)
print("K shape:", K.shape)
print("V shape:", V.shape)

## 3. Scaled Dot-Product Attention (Step by Step)

The core formula:

$$\text{Attention}(Q, K, V) = \text{softmax}\!\left(\frac{Q K^T}{\sqrt{d_k}}\right) V$$

Let's break it into individual steps.

### Step 3a: Raw Attention Scores

$$ \text{scores} = Q \cdot K^T $$

Each element `scores[i][j]` measures how much token `i`'s query aligns with token `j`'s key.

In [ ]:
# Dot product between every pair of (query, key)
scores = Q @ K.T   # (seq_len, seq_len)

print("Raw attention scores shape:", scores.shape)
print(scores)

### Step 3b: Scale

$$ \text{scaled\_scores} = \frac{\text{scores}}{\sqrt{d_k}} $$

Without scaling, large `d_k` pushes dot products to extreme values, making softmax output nearly one-hot (vanishing gradients).

In [ ]:
scale = math.sqrt(d_k)
scaled_scores = scores / scale

print(f"Scale factor: sqrt({d_k}) = {scale:.4f}")
print("Scaled scores:")
print(scaled_scores)

### Step 3c: Softmax (implemented manually)

$$\text{softmax}(x_i) = \frac{e^{x_i}}{\sum_j e^{x_j}}$$

We implement it by hand with the numerical stability trick: subtract the max before exponentiating.

In [ ]:
def softmax(x):
    """Row-wise softmax, implemented from scratch."""
    # Subtract max for numerical stability (prevents overflow in exp)
    x_max = x.max(dim=-1, keepdim=True).values
    exp_x = torch.exp(x - x_max)
    return exp_x / exp_x.sum(dim=-1, keepdim=True)

attn_weights = softmax(scaled_scores)  # (seq_len, seq_len)

print("Attention weights (each row sums to 1):")
print(attn_weights)
print("\nRow sums:", attn_weights.sum(dim=-1))

**Reading the weight matrix:** Row `i` tells you how much each token contributes to the output of token `i`.

For example, if `attn_weights[1] = [0.1, 0.5, 0.3, 0.1]`, then the output for "cat" (token 1) is 50% influenced by itself, 30% by "sat", and 10% each by "The" and "down".

### Step 3d: Weighted Sum of Values

$$\text{output} = \text{attn\_weights} \cdot V$$

Each output row is a weighted combination of all value vectors.

In [ ]:
attn_output = attn_weights @ V  # (seq_len, d_v)

print("Attention output shape:", attn_output.shape)
print(attn_output)

### Putting it all together

In [ ]:
def scaled_dot_product_attention(Q, K, V, mask=None):
    """
    Computes scaled dot-product attention using only basic ops.
    
    Q: (seq_len, d_k)
    K: (seq_len, d_k)
    V: (seq_len, d_v)
    mask: optional (seq_len, seq_len) — True where we want to block attention
    
    Returns: (seq_len, d_v) attention output, (seq_len, seq_len) weights
    """
    d_k = Q.shape[-1]
    
    # 1. Compute scores
    scores = Q @ K.T / math.sqrt(d_k)
    
    # 2. Apply mask (for causal / decoder attention)
    if mask is not None:
        scores = scores.masked_fill(mask, float('-inf'))
    
    # 3. Softmax
    weights = softmax(scores)
    
    # 4. Weighted sum
    output = weights @ V
    
    return output, weights

# Verify it matches our step-by-step result
out, w = scaled_dot_product_attention(Q, K, V)
print("Matches step-by-step?", torch.allclose(out, attn_output, atol=1e-6))

## 4. Causal (Decoder) Mask

In autoregressive models (like GPT), token `i` should only attend to tokens `0..i`, not future tokens. We achieve this with a **causal mask** that sets future positions to `-inf` before softmax.

```
         The  cat  sat  down
The    [  ok  -inf -inf -inf ]
cat    [  ok   ok  -inf -inf ]
sat    [  ok   ok   ok  -inf ]
down   [  ok   ok   ok   ok  ]
```

In [ ]:
# Upper triangular = True where we want to BLOCK attention
causal_mask = torch.triu(torch.ones(seq_len, seq_len, dtype=torch.bool, device=device), diagonal=1)
print("Causal mask (True = blocked):")
print(causal_mask.int())

causal_out, causal_weights = scaled_dot_product_attention(Q, K, V, mask=causal_mask)
print("\nCausal attention weights:")
print(causal_weights)
print("\nNotice: upper triangle is 0 — no attending to future tokens.")

## 5. Multi-Head Attention (from raw ops)

Instead of one attention function, we run `n_heads` in parallel, each with its own Q/K/V projections, then concatenate and project the result.

$$\text{MultiHead}(X) = \text{Concat}(\text{head}_1, \dots, \text{head}_h) \cdot W_O$$

where each head:
$$\text{head}_i = \text{Attention}(X W_Q^i,\; X W_K^i,\; X W_V^i)$$

In [ ]:
n_heads = 2
d_k_per_head = d_model // n_heads  # 4
d_v_per_head = d_model // n_heads  # 4

print(f"n_heads={n_heads}, d_k_per_head={d_k_per_head}, d_v_per_head={d_v_per_head}")

# Create separate weight matrices for each head — raw tensors
W_Qs = [torch.randn(d_model, d_k_per_head, device=device) * 0.1 for _ in range(n_heads)]
W_Ks = [torch.randn(d_model, d_k_per_head, device=device) * 0.1 for _ in range(n_heads)]
W_Vs = [torch.randn(d_model, d_v_per_head, device=device) * 0.1 for _ in range(n_heads)]

# Output projection: maps concatenated heads back to d_model
W_O = torch.randn(n_heads * d_v_per_head, d_model, device=device) * 0.1

In [ ]:
# Run each head independently
head_outputs = []

for i in range(n_heads):
    Q_i = X @ W_Qs[i]  # (seq_len, d_k_per_head)
    K_i = X @ W_Ks[i]
    V_i = X @ W_Vs[i]
    
    head_out, head_weights = scaled_dot_product_attention(Q_i, K_i, V_i)
    head_outputs.append(head_out)
    
    print(f"Head {i} attention weights:")
    print(head_weights)
    print()

In [ ]:
# Concatenate all heads along the last dimension
concat = torch.cat(head_outputs, dim=-1)  # (seq_len, n_heads * d_v_per_head)
print("Concatenated shape:", concat.shape)

# Final linear projection (raw matmul, no nn.Linear)
multi_head_output = concat @ W_O  # (seq_len, d_model)
print("Multi-head output shape:", multi_head_output.shape)
print(multi_head_output)

**Why multiple heads?** Each head can learn to attend to different types of relationships. For instance, one head might focus on adjacent tokens (local syntax), while another head attends to the subject of the sentence (long-range dependency).

## 6. Multi-Head Attention (Batched / Efficient Version)

The loop above is clear but slow. In practice, we pack all heads into a single large projection, then reshape. Still no `nn.Module` — just reshape tricks.

In [ ]:
def multi_head_attention(X, W_Q, W_K, W_V, W_O, n_heads, mask=None):
    """
    Efficient multi-head attention using reshape instead of loops.
    
    X:    (seq_len, d_model)
    W_Q:  (d_model, d_model)  — all heads packed into one matrix
    W_K:  (d_model, d_model)
    W_V:  (d_model, d_model)
    W_O:  (d_model, d_model)
    """
    seq_len, d_model = X.shape
    d_k = d_model // n_heads
    
    # 1. Project all heads at once
    Q = X @ W_Q  # (seq_len, d_model)
    K = X @ W_K
    V = X @ W_V
    
    # 2. Reshape to (n_heads, seq_len, d_k) — split d_model into heads
    Q = Q.view(seq_len, n_heads, d_k).transpose(0, 1)  # (n_heads, seq_len, d_k)
    K = K.view(seq_len, n_heads, d_k).transpose(0, 1)
    V = V.view(seq_len, n_heads, d_k).transpose(0, 1)
    
    # 3. Scaled dot-product attention (batched over heads)
    scores = Q @ K.transpose(-2, -1) / math.sqrt(d_k)  # (n_heads, seq_len, seq_len)
    
    if mask is not None:
        scores = scores.masked_fill(mask.unsqueeze(0), float('-inf'))
    
    weights = softmax(scores)  # softmax works because we wrote it for dim=-1
    attn_out = weights @ V     # (n_heads, seq_len, d_k)
    
    # 4. Concatenate heads: transpose back and reshape
    attn_out = attn_out.transpose(0, 1).contiguous().view(seq_len, d_model)
    
    # 5. Output projection
    output = attn_out @ W_O  # (seq_len, d_model)
    
    return output, weights


# Create packed weight matrices (all heads in one tensor)
W_Q_packed = torch.randn(d_model, d_model, device=device) * 0.1
W_K_packed = torch.randn(d_model, d_model, device=device) * 0.1
W_V_packed = torch.randn(d_model, d_model, device=device) * 0.1
W_O_packed = torch.randn(d_model, d_model, device=device) * 0.1

mha_output, mha_weights = multi_head_attention(
    X, W_Q_packed, W_K_packed, W_V_packed, W_O_packed, n_heads
)

print("MHA output shape:", mha_output.shape)
print("Attention weights shape:", mha_weights.shape, " — (n_heads, seq_len, seq_len)")
print("\nHead 0 weights:")
print(mha_weights[0])
print("\nHead 1 weights:")
print(mha_weights[1])

## 7. Visualizing Attention Weights

In [ ]:
tokens = ["The", "cat", "sat", "down"]

fig, axes = plt.subplots(1, n_heads, figsize=(5 * n_heads, 4))
if n_heads == 1:
    axes = [axes]

for i, ax in enumerate(axes):
    w = mha_weights[i].detach().cpu().numpy()
    im = ax.imshow(w, cmap='Blues', vmin=0, vmax=1)
    ax.set_xticks(range(len(tokens)))
    ax.set_yticks(range(len(tokens)))
    ax.set_xticklabels(tokens)
    ax.set_yticklabels(tokens)
    ax.set_xlabel("Key (attending to)")
    ax.set_ylabel("Query (token)")
    ax.set_title(f"Head {i}")
    
    # Annotate cells with values
    for row in range(len(tokens)):
        for col in range(len(tokens)):
            ax.text(col, row, f"{w[row, col]:.2f}",
                    ha='center', va='center', fontsize=10)

plt.tight_layout()
plt.suptitle("Multi-Head Attention Weights", y=1.02, fontsize=14)
plt.show()

## 8. Summary

Everything we built uses only these primitive operations:

| Operation | PyTorch op | Purpose |
|-----------|-----------|----------|
| Matrix multiply | `@` / `torch.matmul` | Project Q, K, V; compute scores; weighted sum |
| Division | `/` | Scale scores by √d_k |
| Exp | `torch.exp` | Part of softmax |
| Sum | `.sum()` | Part of softmax |
| Max | `.max()` | Numerical stability in softmax |
| Masked fill | `.masked_fill()` | Causal masking |
| Reshape | `.view()`, `.transpose()` | Splitting/merging heads |

No `nn.Module`, no `nn.Linear`, no `nn.MultiheadAttention` — just tensors and math.